---
title: "LC121 and LC53: Kadane's Algorithm"
author: "Vahram Poghosyan"
date: "2024-05-23"
categories: ["Leetcode", "Algorithms", "Dyanmic Progamming"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Problem Statement

We are given an array of `prices` where `prices[i]` is the price of a given stock on the `i`-th day.

We want to maximize our profit by choosing a single day to buy one stock and choosing a different day in the future to sell that stock. Return the maximum profit you can achieve from this transaction. If you cannot achieve any profit, return `0`.

**Example 1**

```
Input: prices = [7,1,5,3,6,4]
Output: 5
```

**Explanation** 

Buy on day 2 (price = 1) and sell on day 5 (price = 6), profit = 6-1 = 5.
Note that buying on day 2 and selling on day 1 is not allowed because you must buy before you sell

**Example 2:**

```
Input: prices = [7,6,4,3,1]
Output: 0
```

**Explanation** 

In this case, no transactions are done and the max profit = 0.

## Brute-Force Solution

Consider each viable pair of days. It's easy to see that this leads to time complexity $O(n^2)$ because, for each possible day `i` that we choose to buy the stock on, there are $n-i$ possible days that we can sell it on. Since there are $n$ choices for which day to buy, the number of total pairs has a leading term of $n(n-1)$, so it's quadratic in $n$. 

We can also think of choosing a subset of size $2$ and discarding those which have a reverse order of days. This essentially means choosing a subset of size $2$ without order (since each pair is either in the correct order or not, and we only count the one that is), so ${O \left ({n \choose 2} \right )}$ which is, of course, $O(n^2)$.

## Non-Brute-Force Solution

We can solve this problem in a single pass, achieving $O(n)$ complexity by using DP with tabulation similar to [Kadane's algorithm](https://en.wikipedia.org/wiki/Maximum_subarray_problem#Kadane's_algorithm) which solves the [Maximum Subarray](https://en.wikipedia.org/wiki/Maximum_subarray_problem) problem. The similarities between these two problems are due to fact that both are concerned with some score over a *contiguous* array (contiguity being what gives rise to the optimal substructure). Whereas Kadane's is concerned with the contiguous subarray with maximum sum, this algorithm is interested in the maximum profit (which is the difference between the last element of the optimal subarray and the first one). Like Kadane's algorithm, we can prove its correctness using *loop invariants*. We will give the solution and prove the correctness. As for building intuition for why this solution works, we will focus on the [Maximum Subarray problem](https://en.wikipedia.org/wiki/Maximum_subarray_problem) which is a more general application of this type of pattern.  

### Code

In [1]:
#| code-fold: false
def maxProfit(prices):
    min_price = float("inf") # +infinity
    max_profit = 0 

    for i in range(len(prices)):
        if prices[i] < min_price:
            min_price = prices[i]
        elif prices[i] - min_price > max_profit:
            max_profit = prices[i] - min_price

    return max_profit

prices = [7,1,5,3,6,4]
print(maxProfit(prices))

5


## Proof of Correctness

It's easy to see that, at the end of each iteration `i`, the variable `min_price` holds the lowest price dip in the stock up to, and including, the index `i`. This is called a loop invariant. Showing something is a loop invariant is a lot like proving the *inductive step* in a proof by induction (for example, when trying to prove recursive algorithms).

It can also be shown that `max_profit` is another loop invariant, and that it holds the maximum profit up to, and including, the index `i`. 

### Proof of Invariance

The loop does one of two things exclusively: either it updates `min_price` or it doesn't. 

Suppose it *doesn't* update `min_price` (**case 1**). The first loop invariant, `min_price` holds the lowest price dip up to, and including, the index `i`. In this case, the difference of `prices[i]` and `min_price` is then calculated and `max_profit` is updated *only* if the difference is greater than the `max_profit` at the end of the previous iteration (iteration `i-1`). This guarantees that `max_profit` holds the maximum profit up to, and including, the index `i`. 

In the other case (**case 2**), when the loop *does* update `min_price]` at iteration `i`, it enters the subsequent iteration `i+1` with `max_profit` still holding the maximum profit up to, and including, index `i` (at the beginning of the iteration). If `prices[i+1]` is, again, less than `min_price` then the loop just goes on updating `min_price` *only* until it encounters the lowest price dip (unless the price keeps dipping until the very end, in which case the proof is complete). At this iteration of the loop, let's call it `k+1`, `max_profit` still holds the maximum profit up to, and including, index `k` (because there have just been consecutive dips in price since index `i`). Since `prices[k+1]` is the lowest price dip, by assumption, in the next iteration we are necessarily in the familiar again (**case 1**). Hence, `max_profit` is a loop invariant. 

Therefore, once the loop is finished, `max_profit` will hold the maximum profit up to, and including, the last index `n`. In other words, it will hold the solution to the problem. 

## Kadane's Algorithm - Maximum Subarray

### Dynamic Programming - Key Idea

The key idea behind dynamic programming is to solve sub-problems once, store their results one way or another, and use them to solve the larger problem. Usually there's some an optimal substructure to the problems which lend themselves to a solution by DP that allows us to obtain the solution to the larger problem from that of its sub-problems using a computationally cheap trick.  

#### Two Approaches

| Approach | Explanation |
|----------|-------------|
| **Top-Down (Memoization)**   | Uses **recursion** to solve the problem, and **memoization** to store the solutions to sub-problems |
| **Bottom-Up (Tabulation)**:   | **Iteratively** solves sub-problems in a specific order, eliminating the need for recursion (this technique is also known as **tabulation**)|

### Applying DP to the Maximum Sub-Array Problem

Kadane's algorithm, which solves this problem in a single pass, relies on an optimal substructure in the problem. This optimal substructure that comes from the contiguity requirement. It's what allows us to reduce the total number of passes required over the sub-arrays when computing their sums. The sub-problems are solved in a particular order, such that the solution of one sub-problem, stored in memory, obtains the solution to the next sub-problem and so on.

First, let's break the overall problem down into its constituent sub-problems. There are several ways of doing that.

One way is to observe that any subarray ends (or begins) at some index `k`. We may define an abstraction such as this: The solution to the problem of size-`k`. Note that this definition isn't pulled out of thin air, it's the global maximum for the `k`-sized sub-problem (i.e. the solution to the overall problem were it to have size `k`). So, let's call it `global_max(k)`.

We may define yet another useful abstraction: The solution to the problem if the optimal sub-array was constrained to those sub-arrays which end at index `k`. Note that this, also, isn't pulled out of thin air. It's the local maximum at index `k`. So, we call it `local_max(k)`.

Note that `global_max(k)` and `local_max(k)` aren't the same thing. It's easy to get lost in trying to solve this problem by conflating `local_max` with `global_max`, but they're not the same. They're also the only variables needed to implement a single-pass solution, as we'll soon see. Let's see how these two definitions differ.

There may be an input array for which the best `local_max(k)` can achieve are negative sums. For instance, take `k=3` in the following Manim animation (which demonstrates such a case).


::: {#fig-kadane-manim}
![Kadane's Demo](./media/videos/leetcode/720p30/KadaneAlgoDemo.mp4)
:::

In the case above, the `global_max(k)` actually retains its previous value (`global_max[k-1]`), totally omitting the `k`-th cell from consideration. But the word *Sum* in the video is closely related to `local_max[k]` -- for a given index `k`, `local_max[k]` is the greatest of the *Sum* values for that index. This spells out the following optimal substructure:


```
global_max[k] = max(global_max[k-1], global_max[k-1] + local_max[k])
```

This is a greedy-choice update for `global_max`. It gives us a shortcut by which to update `global_max[k]`, if we already had `local_max[k]` for free (by some dark magic). If we had a way to go from `local_max[k-1]` to `local_max[k]`, we'd get to the final solution very quickly. We would start with `global_max[0]`, the base sub-problem whose solution is, trivially, `nums[0]`, and plug away in a single pass until we got to `global_max[n]` (for some `n`-sized problem). Luckily for us, there *is* such a cheap update for `local_max[k]` based on `local_max[k-1]`. It is, again, the result of an optimal sub-structure (stemming from the contiguity requirement).

For any given index `k`, to obtain `local_max[k]`, we need not compute the *Sums* of all the sub-arrays ending at `k`. Since we know `local_max[k-1]` from solving the previous sub-problem, the update is simply:

```
local_max[k] = max(local_max[k-1] + nums[k], nums[k])
```

To convince ourselves of this, let's see another Manim video which highlights this optimal sub-structure.

::: {#fig-kadane-manim}
![Kadane's Demo](./media/videos/leetcode/720p30/KadaneAlgoDemo.mp4)
:::

In the video above you’ll notice that any sub-array ending at `k` can be divided into two parts, a sub-array ending at index `k-1` (highlighted in yellow) and the single-element sub-array `nums[k]` (in green). So, since we know `local_max[k-1]`, to find out `local_max[k]` we no longer need to compute the sums of all the possible sub-arrays ending at `k`. Rather, `local_max[k]` will always just be either `nums[k]` or the sum of `nums[k]` and `local_max[k-1]`. So we can do away with a lot of the summation in the brute force approach. 
 
So, let's walk through a potential solution using these two optimal sub-structures. Because we know `nums[0]` and `local_max[0]` (trivially), we can find out `local_max[1]`. And, since we have `global_max[0]` (again, trivially), then we can find `global_max[1]`. That's the solution to the sub-problem of size `1`. It's easy to see that proceeding iteratively in this manner, by first updating `local_max[k]` then `global_max[k]` on some iteration `k`, we can arrive at the solution to the overall `n`-sized problem ()`global_max[n]`) without the need to compute sums over sub-arrays.  

Now we just need to implement a loop that updates `local_max` and `global_max` in this way. For more involved DP problems, we may need a matrix (or some other higher-dimensional data structure) to store the solutions to the sub-problems. But for this problem, two loop invariants (`local_max` and `global_max`) will suffice. Let's write the solution.

In [ ]:
local_max = float('-inf')
global_max = float('-inf')

for i, num in enumerate(nums):
    if num > local_max + num:
        local_max = num
    else:
        local_max = local_max + num
    if global_max < local_max:
        global_max = local_max

In [ ]:
from manim import *

Let's start with illustrating all subarrays that end at an index `i` (by varying this index).

In [ ]:
%%manim -qm KadaneAlgoDemo

from manim import *

class KadaneAlgoDemo(Scene):
    def construct(self):    
        # Create table
        array=[[1,-2,3,-5,2,3]]
        table = IntegerTable(array, include_outer_lines=True)
        self.play(Write(table))
        # Get table out of the way...
        self.wait(1)
        self.play(table.animate.scale(0.25)) 
        self.play(table.animate.to_corner(UP + LEFT))

        # Positioning of subarrays
        h_shift = 6.25
        v_shift = 0.5

        # Print subarray(s) ending at index 0
        index = 0
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i]    
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 1*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 1
        index = 1
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i]           
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 2*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 2
        index = 2
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 3 + 3*0.20
        v_shift = 0.5 # Reset vertical shift

        # Print subarray(s) ending at index 3
        index = 3
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT + 2.5 * UP))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift = 5 # Reset horizontal shift
        v_shift = 3 # Reset vertical shift

        # Print subarray(s) ending at index 4
        index = 4
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

        # Positioning of subarrays
        h_shift -= 5 + 4*0.20
        v_shift = 3 # Reset vertical shift

        # Print subarray(s) ending at index 4
        index = 5
        index_text = Text(f"i = {index}")
        self.play(Write(index_text))
        self.play(index_text.animate.scale(0.35))
        self.play(index_text.animate.shift(h_shift * LEFT))
        for left in range(index+1):
            subarray=[array[0][left:index+1]]
            subtable = IntegerTable(subarray, include_outer_lines=True)
            self.play(Write(subtable))
            self.play(subtable.animate.scale(0.25)) 
            self.play(subtable.animate.shift(h_shift * LEFT + 2.5 * UP + v_shift * DOWN))
            subarray_sum = sum(subarray[0]) # This is local_max[i] 
            subarray_sum_text = Text(f"Sum: {subarray_sum}")
            self.play(Write(subarray_sum_text))
            self.play(subarray_sum_text.animate.scale(0.35))
            self.play(subarray_sum_text.animate.next_to(subtable, RIGHT))
            h_shift -= 0.20
            v_shift += 0.5
        self.wait(1)

::: {#fig-kadane-manim}
![Kadane's Demo](./media/videos/leetcode/720p30/KadaneAlgoDemo.mp4)
:::

The final answer is the maximum over all the local maxima (obviously).

Thinking in terms of solutions to sub-problems is, in essence, what dynamic programming is. It's a way to cut down on the number of subarrays considered in the brute force approach by coming up with a computationally cheap rule that gives us the solution to the current problem based on the retrieved value of a previously solved sub-problem (whatever the relationship between the problem and its sub-problems may be). In the case of the Maximum Subarray problem the sub-problems are expressed in terms of `local_max[i]` and `global_max[i]`. If we know `local_max[i-1]`, `local_max[i]` is all but known through the above relationship and a similar relationship also exists for `global_max`.

If we work through an example, the act itself will give us insight into the implementation of the single pass, iterative algorithm. Note that the recursive approach is already somewhat betrayed by the optimal substructure and, to further make it dynamic, we could implement memoization as a layer above it. Let's take the first step, and then generalize.

In the beginning, there's just `nums[0]`. By virtue of being the only subarray that ends at index `0` it is its own `local_max[0]`. In situations like this we initialize `local_max` to $-\infty$ in order not to resort to handling special cases, like singleton arrays, in the loop.

```python
local_max = float('-inf')

for i, num in enumerate(nums):
    if num > local_max:
        local_max = num
```

It's immediately clear that, since `local_max` is initialized to $-\infty$ (let's call this `local_max[-1]` to stay consistent with array notation of the optimal substructure), `num > local_max` is equivalent to:

```
nums[0] > local_max[-1] + nums[0]
```

This is true in general for any `num[0]`. So the desired condition is actually: 

```python
local_max = float('-inf')

for i, num in enumerate(nums):
    if num > local_max + num:
        local_max = num
    else
        local_max = local_max + num
```

As it is, the loop invariant `local_max` will contain the local maximum at the last element of the `nums` array. We need to figure out what to do with the other loop invariant `global_max` which is supposed to be the solution to the problem of size `i`. Well, `global_max` is clearly just: 

```
global_max[i] = max(global_max[i-1], local_max[i])
```
Which is the promised greedy-choice update for `global_max` -- the loop invariant which will hold the final solution to the problem.

We need to implement this relationship just as we implemented the previous one unless we just use the built in `max` function to essentially just update both loop invariants in sequence and be done with it. But let's go with the more imperative version. Let's take the first step. 

In the beginning there's just `nums[0]`. Clearly `global_max[0]` equals `nums[0]` as that's what `local_max[0]` is and `global_max[-1]` (as per our earlier abuse of notation) is $-\infty$ by choice.

```python
local_max = float('-inf')
global_max = float('-inf')

for i, num in enumerate(nums):
    if num > local_max + num:
        local_max = num
    else
        local_max = local_max + num
    if global_max < local_max:
        global_max = local_max
```

We can reference `local_max` and assume its value to be as the promised invariant on line 9. 

Notice that `global_max` has two meanings, as does `local_max` in the implementation. It is used as the previous iterate `global_max[i-1]` (in the Boolean comparison) as well as the current (or next, depending on frame of reference) iterate `global_max[i]` (in the assignment operation).